In [1]:
import lib.facebook_scraper as fs
import lib.utils as utils
import pandas as pd
import numpy as np
import os
import time

In [2]:
post_ids_df = pd.read_csv('data/vebay69/clean/post_ids.csv')
post_ids_df.head()

,post_id
0,685638947004646
1,685577330344141
2,685111467057394
3,684686633766544
4,684603707108170


In [3]:
NUMBER_POST = 10
post_list = []
sharer_list = []
total_sharer_real = 0
index_post_id_file_path = os.path.join('data','vebay69','url','i_post_id_share.txt')

In [4]:
index_post_id = utils.read_index_file(file_path=index_post_id_file_path)
print(index_post_id)
posts_ids = utils.get_post_id_list(df=post_ids_df, start=index_post_id, length=NUMBER_POST)
posts_ids

0


[685638947004646,
 685577330344141,
 685111467057394,
 684686633766544,
 684603707108170,
 684191557149385,
 683955240506350,
 682996903935517,
 683571943878013,
 683467500555124]

In [5]:
def extract_sharer(sharers, post_id):
    cnt_sharer = 0
    global sharer_list
    for sharer in sharers:
        sharer_list.append(sharer)
        cnt_sharer += 1
        time.sleep(np.random.uniform(0, 1))
    return cnt_sharer

In [6]:
fs.set_cookies('./cookies/cookies_1.txt')
# fs.enable_logging()

In [7]:
try:
    for post in fs.get_posts(
        post_urls=posts_ids,
        options={
            "allow_extra_requests": True,
            "sharers": True,
            "sharers": "generator",
        },
        timeout=120,
    ):
        print(post)
        post_list.append(post)
        num_sharer = extract_sharer(post['sharers'], post['original_request_url'])
        print(f'Extracted {num_sharer}/{post["shares"]} sharers')
        total_sharer_real += post['shares']
        utils.sleep(np.random.randint(10, 30))
        print()
except fs.exceptions.TemporarilyBanned:
    print("TEMPORARILY BANNED")
except fs.exceptions.AccountDisabled:
    print("ACCOUNT DISABLED")
except Exception as e:
    print(e)

/home/ad/Workspace/data-processing/analyze-facebook-page/lib/facebook_scraper/facebook_scraper.py:912: UserWarning: Facebook says 'Unsupported Browser'
  warnings.warn(f"Facebook says 'Unsupported Browser'")


{'original_request_url': 685638947004646, 'post_url': 'https://facebook.com/story.php?story_fbid=685638947004646&id=100066756416622', 'post_id': 685638947004646, 'text': 'Mùa đông chưa vậy\n#Panda', 'post_text': 'Mùa đông chưa vậy\n#Panda', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2023, 11, 20, 12, 28, 2), 'timestamp': 1700458082, 'image': 'https://scontent.fhan14-1.fna.fbcdn.net/v/t39.30808-6/404513275_685638733671334_3197628116849525623_n.jpg?stp=cp0_dst-jpg_e15_fr_q65&_nc_cat=107&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_eui2=AeHzPzLflDVbDQ8Biie60FAskkYLACx62iSSRgsALHraJHFQc-pyjEqXtPILCLfY5-JI8S19RD9RdJlLxMUVJbsu&_nc_ohc=E9eLc0YDr0QAX-CKk4C&_nc_ht=scontent.fhan14-1.fna&oh=00_AfAjlSDGiBwLxVja_ytjwDVEZrjHR3WuOsQKJP0x9xiQbg&oe=6565C85D&manual_redirect=1', 'image_lowquality': 'https://scontent.fhan14-1.fna.fbcdn.net/v/t39.30808-6/404513275_685638733671334_3197628116849525623_n.jpg?stp=cp0_dst-jpg_e15_p320x320_q65&_nc_cat=107&ccb=1-7&_nc_sid=5f2048&efg=eyJpIj

In [8]:
print(NUMBER_POST)
print(f"Số posts đã crawl: {len(post_list)}")
print(f"Số sharers đã crawl: {len(sharer_list)}/{total_sharer_real}")

10
Số posts đã crawl: 10
Số sharers đã crawl: 440/862


In [10]:
if sharer_list:
    utils.save_data(
        data_list=sharer_list,
        folder_path=os.path.join('data','vebay69','raw'),
        type="sharers"
    )
utils.write_index_file(
    file_path=index_post_id_file_path,
    index=index_post_id + len(post_list)
)

Save sharers data: ./data/vebay69/raw/sharers_2023-11-24_15-20-34.csv
Save index 10 to resume: ./data/vebay69/url/i_post_id_share.txt
